In [1]:
from pathlib import Path

import pandas as pd
from pyproj import CRS

# from bedrock.gi.ags.read import ags_to_dfs
# from bedrock.gi.ags.transform import ags3_db_to_no_gis_brgi_db
from bedrock.gi.concatenate import concatenate_databases
from bedrock.gi.gis_geometry import calculate_gis_geometry
from bedrock.gi.write import write_gi_db_to_gpkg

# pd.set_option("display.max_rows", None)
# pd.set_option("display.max_columns", None)

In [2]:
cwd = Path.cwd()
gi_dir = cwd.parent / "data" / "leapfrog" / "weka-hills"
gi_projects = sorted(list(gi_dir.glob("*")))
gi_projects

[WindowsPath('c:/Users/joost/ReposWindows/bedrock-gi/data/leapfrog/weka-hills/boreholes'),
 WindowsPath('c:/Users/joost/ReposWindows/bedrock-gi/data/leapfrog/weka-hills/cpt')]

In [3]:
class MissingTableError(Exception):
    pass


def assign_location_uid(df):
    # Search for the column containing 'locationid' or 'holeid'
    for col in df.columns:
        if "locationid" in col.lower() or "holeid" in col.lower():
            df["location_uid"] = df[col]  # Create the 'location_uid' column
            break  # Stop searching after finding the first match

In [4]:
crs_wkt = CRS(2193).to_wkt()
brgi_db = {}
for gi_project in gi_projects:
    project_uid = f"{gi_project.parent.name}-{gi_project.name}"
    brgi_db_from_1_project = {}
    brgi_db_from_1_project["Project"] = pd.DataFrame(
        {
            "project_uid": [f"{gi_project.parent.name}-{gi_project.name}"],
            "crs_wkt": [crs_wkt],
        }
    )
    gi_table_paths = sorted(list(gi_project.glob("*.csv")))

    location_table_names = set(["location", "collar", "hole", "borehole", "well"])
    gi_tables_lower = [gi_table.stem.lower() for gi_table in gi_table_paths]
    if not location_table_names & set(gi_tables_lower):
        raise MissingTableError(
            'A table with GI Location data is missing. This table should be called "location", "collar", "hole", "borehole" or "well".'
        )

    for table_path in gi_table_paths:
        table_name = table_path.stem
        table_data = pd.read_csv(table_path)
        table_data["project_uid"] = project_uid
        table_data["location_uid"] = table_data["LocationID"]

        if table_name.lower() in location_table_names:
            table_data["location_source_id"] = table_data["LocationID"]
            table_data["location_type"] = gi_project.name
            table_data.rename(columns={"Easting": "easting"}, inplace=True)
            table_data.rename(columns={"Northing": "northing"}, inplace=True)
            table_data["ground_level_elevation"] = table_data["GroundLevel"]
            table_data["depth_to_base"] = table_data["FinalDepth"]
            brgi_db_from_1_project["Location"] = table_data
        elif table_name.lower() == "spt":
            table_data["depth_to_top"] = table_data["from"]
            brgi_db_from_1_project["InSitu_SPT"] = table_data
        else:
            if "location_uid" in table_data.columns:
                table_data["depth_to_top"] = table_data["from"]
                table_data["depth_to_base"] = table_data.get("to")
                brgi_db_from_1_project[f"InSitu_{table_name}"] = table_data

    brgi_db = concatenate_databases(brgi_db, brgi_db_from_1_project)

In [5]:
brgi_db = calculate_gis_geometry(brgi_db)

Calculating GIS geometry for the Bedrock GI database tables...
Calculating GIS geometry for the Bedrock GI 'Location' table...
Creating 'LonLatHeight' table with GI locations in WGS84 geodetic coordinates...
    WGS84 geodetic coordinates: (Longitude, Latitude, Ground Level Ellipsoidal Height)
Calculating GIS geometry for the Bedrock GI 'InSitu_Geol' table...
Calculating GIS geometry for the Bedrock GI 'InSitu_SPT' table...
Calculating GIS geometry for the Bedrock GI 'InSitu_CPT' table...


In [6]:
brgi_db.keys()

dict_keys(['Project', 'InSitu_Geol', 'Location', 'InSitu_SPT', 'InSitu_CPT', 'LonLatHeight'])

In [7]:
brgi_db["Project"]

,project_uid,crs_wkt
0,weka-hills-boreholes,"PROJCRS[""NZGD2000 / New Zealand Transverse Mer..."
1,weka-hills-cpt,"PROJCRS[""NZGD2000 / New Zealand Transverse Mer..."


In [10]:
brgi_db["InSitu_SPT"].head()

,LocationID,from,to,SPT,Rock Quality,project_uid,location_uid,depth_to_top,elevation_at_top,geometry
0,WH_016,0.0,1.0,11.0,"Limestone: V, VI",weka-hills-boreholes,WH_016,0.0,610.18681,POINT Z (1165762.475 4888326.462 610.187)
1,WH_016,1.0,2.0,3.0,"Limestone: V, VI",weka-hills-boreholes,WH_016,1.0,609.18681,POINT Z (1165762.475 4888326.462 609.187)
2,WH_016,2.0,3.0,2.0,"Limestone: V, VI",weka-hills-boreholes,WH_016,2.0,608.18681,POINT Z (1165762.475 4888326.462 608.187)
3,WH_016,3.0,4.0,6.0,"Limestone: V, VI",weka-hills-boreholes,WH_016,3.0,607.18681,POINT Z (1165762.475 4888326.462 607.187)
4,WH_016,4.0,5.0,9.0,"Limestone: V, VI",weka-hills-boreholes,WH_016,4.0,606.18681,POINT Z (1165762.475 4888326.462 606.187)


In [9]:
len(brgi_db["InSitu_SPT"])

5029

In [11]:
brgi_db["InSitu_Geol"].to_file(cwd / "weka_hills_gi_Geol.geojson", driver="GeoJSON")

In [12]:
write_gi_db_to_gpkg(brgi_db, cwd / "weka_hills_gi.gpkg")

Ground Investigation data has been written to 'c:\Users\joost\ReposWindows\bedrock-gi\sandbox\weka_hills_gi.gpkg'.
